<a href="https://colab.research.google.com/github/alessiomongoli/Sentiment_Lexicon/blob/main/Utils/DataframeCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
def creation_dataframe(vocabulary,coefficents, frequencies):
  word=[]
  valore=[]
  pol=[]
  for key,value in vocabulary.items():
    word.append(key)
    valore.append(frequencies[value])
    pol.append(coefficents[value])
  data=pd.DataFrame(columns=['Token','Frequence','Polarity'])
  data['Token']=word
  data['Frequence']=valore
  data['Polarity']=pol
  return data